# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

## Drive setup

In [2]:
!ls

datalab


In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
# !ls drive/models/research/object_detection/test_images
!ls drive

App					cropped_eyes_training_scores.npy
App v1					eye_dataset
Copy of cifar10_inception_solver.ipynb	Eyes.ipynb
cropped_eyes_testing.npy		Gazecapture_Dataset
cropped_eyes_testing_scores.npy		models
cropped_eyes_training.npy		Untitled0.ipynb


In [5]:
import os

!apt-get install protobuf-compiler

os.chdir('/content/drive/models/research')
# # From tensorflow/models/research/
!protoc object_detection/protos/*.proto --python_out=.

os.chdir('/content')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libprotobuf10 libprotoc10
The following NEW packages will be installed:
  libprotobuf10 libprotoc10 protobuf-compiler
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 1,239 kB of archives.
After this operation, 4,935 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/main amd64 libprotobuf10 amd64 3.0.0-9ubuntu5 [650 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful/main amd64 libprotoc10 amd64 3.0.0-9ubuntu5 [565 kB]
Get:3 http://archive.ubuntu.com/ubuntu artful/main amd64 protobuf-compiler amd64 3.0.0-9ubuntu5 [24.5 kB]
Fetched 1,239 kB in 0s (1,461 kB/s)
Selecting previously unselected package libprotobuf10:amd64.
(Reading database ... 18149 files and directories currently installed.)
Preparing to unpack .../libprotobuf10_3.0.0-9ubuntu5_amd64.deb ...
Unpacking libprotobuf

In [6]:
import os

!echo $PYTHONPATH
!pwd

os.environ['PYTHONPATH'] = '/env/python' + ':/content/drive/models/research/' + ':/content/drive/models/research/slim' + ':/content/drive/models/research/object_detection'

!echo $PYTHONPATH

/env/python
/content
/env/python:/content/drive/models/research/:/content/drive/models/research/slim:/content/drive/models/research/object_detection


# Imports

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/content/drive/models/research/")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [0]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [9]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/content/drive/models/research/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [0]:
# What model to download.
MODEL_NAME = 'drive/models/research/object_detection/face_detection_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('drive/models/research/object_detection/data', 'object-detection.pbtxt')

NUM_CLASSES = 1

## Download Model

## Load a (frozen) Tensorflow model into memory.

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [0]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [0]:
import glob, os
from PIL import Image
import numpy as np

# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'drive/eye_dataset/Columbia_Gaze_Data_Set'

IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '000{}'.format(i)) for i in range(1, 10) ]
IMAGE_PATHS.extend([ os.path.join(PATH_TO_TEST_IMAGES_DIR, '00{}'.format(i)) for i in range(10, 57) ])
TEST_IMAGE_PATHS = []

real_images = []

# for i in range(56): # 56
#     for file in glob.glob(os.path.join(IMAGE_PATHS[i], "00*_0P*")):
#         TEST_IMAGE_PATHS.append(file)
        
#         image = Image.open(file)
#         image_np = load_image_into_numpy_array(image)

#         real_images.append(image_np);
        
# print(TEST_IMAGE_PATHS)
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [15]:
!ls drive

App					cropped_eyes_training_scores.npy
App v1					eye_dataset
Copy of cifar10_inception_solver.ipynb	Eyes.ipynb
cropped_eyes_testing.npy		Gazecapture_Dataset
cropped_eyes_testing_scores.npy		models
cropped_eyes_training.npy		Untitled0.ipynb


In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
#       print(all_tensor_names)
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [34]:
import glob, os
from PIL import Image
import numpy as np
from scipy.misc import imresize
import re

cropped_eyes_training = []
cropped_eyes_testing = []
y_values_training = []
y_values_testing = []
filenames_test = []

for i in range(56): # 56
    print(str(int((i+1)/56*100)) + '% ' + str(i+1) + '/56')
    
    current_files = glob.glob(os.path.join(IMAGE_PATHS[i], "00*_0P*"))
    
    learning_indexes = np.random.choice(len(current_files), int(85/100*len(current_files)), replace=False)
    testing_indexes = list(set(range(len(current_files))).difference(learning_indexes))
    
    learning_files = list(np.asarray(current_files)[learning_indexes])
    testing_files = list(np.asarray(current_files)[testing_indexes])
    
    # Training set
    for file in learning_files:
        image = Image.open(file)
        image_np = load_image_into_numpy_array(image)

        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        output_dict = run_inference_for_single_image(image_np, detection_graph)

        maxValIndex = np.where(output_dict['detection_scores'] == output_dict['detection_scores'].max())[0]
        bounds = output_dict['detection_boxes'][maxValIndex][0]
        bounds[0] = bounds[0]*np.size(image_np, 0) # y
        bounds[1] = bounds[1]*(np.size(image_np, 1) - 1/10) # x
        bounds[2] = bounds[2]*np.size(image_np, 0) # h
        bounds[3] = bounds[3]*(np.size(image_np, 1) + 1/10) # w

        bounds = bounds.astype(int)
        
        try:
          extracted_img = image_np[int(10/8*bounds[0]):int(bounds[2] - 5/9*(bounds[2]-bounds[0])), \
                                          int(bounds[1]):int(bounds[3])]
          parsed_name = re.compile('[_\.]').split(file)
          current_score = [int(parsed_name[-3][:-1]), int(parsed_name[-2][:-1])]
          
          cropped_eyes_training.append(imresize(extracted_img, 1/4))
          y_values_training.append(current_score)
        except Exception as e:
          print('Error on ' + file)
          print('Last file: ' + file)
  
    # Testing set
    for file in testing_files:
        image = Image.open(file)
        image_np = load_image_into_numpy_array(image)

        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        output_dict = run_inference_for_single_image(image_np, detection_graph)

        maxValIndex = np.where(output_dict['detection_scores'] == output_dict['detection_scores'].max())[0]
        bounds = output_dict['detection_boxes'][maxValIndex][0]
        bounds[0] = bounds[0]*np.size(image_np, 0) # y
        bounds[1] = bounds[1]*(np.size(image_np, 1) - 1/10) # x
        bounds[2] = bounds[2]*np.size(image_np, 0) # h
        bounds[3] = bounds[3]*(np.size(image_np, 1) + 1/10) # w

        bounds = bounds.astype(int)
        
        try:
          extracted_img = image_np[int(10/8*bounds[0]):int(bounds[2] - 5/9*(bounds[2]-bounds[0])), \
                                          int(bounds[1]):int(bounds[3])]
          parsed_name = re.compile('[_\.]').split(file)
          current_score = [int(parsed_name[-3][:-1]), int(parsed_name[-2][:-1])]
          
          cropped_eyes_testing.append(imresize(extracted_img, 1/4))
          y_values_testing.append(current_score)
        except Exception as e:
          print('Error on ' + file)
        
        
#         for i in range(3):
#           for j in range(3):
#             cropped_eyes_list.append(image_np[int(10/8*bounds[0] + 1.5*60 - j*60):int(bounds[2] - 5/9*(bounds[2]-bounds[0]) + 1.5*60 - j*60), \
#                                               int(bounds[1] + 1.5*60 - i*60):int(bounds[3] + 1.5*60 - i*60)])

      
# for image_path in TEST_IMAGE_PATHS:
#   print(image_path)
#   image = Image.open(image_path)
#   # the array based representation of the image will be used later in order to prepare the
#   # result image with boxes and labels on it.
#   image_np = load_image_into_numpy_array(image)
#   # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
#   image_np_expanded = np.expand_dims(image_np, axis=0)
#   # Actual detection.
#   output_dict = run_inference_for_single_image(image_np, detection_graph)
  
# #   Visualization of the results of a detection.
# #   vis_util.visualize_boxes_and_labels_on_image_array(
# #       image_np,
# #       output_dict['detection_boxes'],
# #       output_dict['detection_classes'],
# #       output_dict['detection_scores'],
# #       category_index,
# #       instance_masks=output_dict.get('detection_masks'),
# #       use_normalized_coordinates=True,
# #       line_thickness=8)
  
#   maxValIndex = np.where(output_dict['detection_scores'] == output_dict['detection_scores'].max())[0]
#   bounds = output_dict['detection_boxes'][maxValIndex][0]
#   bounds[0] = bounds[0]*np.size(image_np, 0) # y
#   bounds[1] = bounds[1]*(np.size(image_np, 1) - 1/10) # x
#   bounds[2] = bounds[2]*np.size(image_np, 0) # h
#   bounds[3] = bounds[3]*(np.size(image_np, 1) + 1/10) # w
  
#   bounds = bounds.astype(int)
#   print(bounds)
  
#   for i in range(5):
#     for j in range(3):
#       cropped_eyes_list.append(image_np[int(10/8*bounds[0] + 1.5*60 - j*60):int(bounds[2] - 5/9*(bounds[2]-bounds[0]) + 1.5*60 - j*60), \
#                                         int(bounds[1] + 3*30 - i*30):int(bounds[3] + 3*30 - i*30)])
      
#   for i in range(15):
#     plt.figure(figsize=IMAGE_SIZE)
#     plt.axis('off')
#     plt.imshow(cropped_eyes_list[i])
      
# #   print(output_dict['detection_scores'])
  

1% 1/56


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


3% 2/56
5% 3/56
7% 4/56
8% 5/56
10% 6/56
12% 7/56
14% 8/56
16% 9/56
17% 10/56
19% 11/56
21% 12/56
23% 13/56
25% 14/56
26% 15/56
28% 16/56
30% 17/56
32% 18/56
Error on drive/eye_dataset/Columbia_Gaze_Data_Set/0018/0018_2m_0P_0V_5H (7761c5a2).jpg
Last file: drive/eye_dataset/Columbia_Gaze_Data_Set/0018/0018_2m_0P_0V_5H (7761c5a2).jpg
Error on drive/eye_dataset/Columbia_Gaze_Data_Set/0018/0018_2m_0P_-10V_10H (fab278ef).jpg
Last file: drive/eye_dataset/Columbia_Gaze_Data_Set/0018/0018_2m_0P_-10V_10H (fab278ef).jpg
33% 19/56
35% 20/56
37% 21/56
Error on drive/eye_dataset/Columbia_Gaze_Data_Set/0021/0021_2m_0P_-10V_-15H.jpg
Last file: drive/eye_dataset/Columbia_Gaze_Data_Set/0021/0021_2m_0P_-10V_-15H.jpg
39% 22/56
41% 23/56
42% 24/56
44% 25/56
46% 26/56
48% 27/56
50% 28/56
51% 29/56
53% 30/56
55% 31/56
57% 32/56
58% 33/56
60% 34/56
62% 35/56
64% 36/56
66% 37/56
67% 38/56
69% 39/56
71% 40/56
73% 41/56
75% 42/56
76% 43/56
78% 44/56
80% 45/56
82% 46/56
83% 47/56
85% 48/56
87% 49/56
89% 50/56
91

In [0]:
np.save('/content/drive/cropped_eyes_training.npy', cropped_eyes_training)
np.save('/content/drive/cropped_eyes_training_scores.npy', y_values_training)

np.save('/content/drive/cropped_eyes_testing.npy', cropped_eyes_testing)
np.save('/content/drive/cropped_eyes_testing_scores.npy', y_values_testing)

In [0]:
# !ls

# test_vect = np.arange(20).reshape(2,2,5)
# print(test_vect)
# test_vect2 = []
# # np.savetxt('/content/drive/test_vect.npy', test_vect)
# # np.loadtxt('/content/drive/test_vect.npy', test_vect2)
# np.save('/content/drive/test_vect.npy', test_vect)
# test_vect2 = np.load('/content/drive/test_vect.npy')

# print(test_vect2)

In [0]:
# SHow images
# for i in range(np.size(cropped_eyes_list, 0)):
#   plt.figure(figsize=IMAGE_SIZE)
#   plt.axis('off')
#   plt.imshow(cropped_eyes_list[i])
#   print(y_values[i])

## Crop

In [1]:
from scipy.misc import imresize

print(bounds)

# print(image_np[bounds[0]:bounds[2]])
# plt.imshow(image_np[int(10/8*bounds[0]):int(bounds[2] - 5/9*(bounds[2]-bounds[0])), bounds[1]:bounds[3]])
# plt.imshow(cropped_eyes_list[485])

x = cropped_eyes_list[190]
# x = imresize(x, 1/4)
# plt.axis('off')
plt.imshow(x)

NameError: ignored

1


In [85]:
import re

for file in glob.glob(os.path.join(IMAGE_PATHS[i], "00*_0P*")):
  y_value = re.compile('[_\.]').split(file)
  y_value = [int(y_value[-3][:-1]), int(y_value[-2][:-1])]
  print(y_value)

[0, 15]
[10, -10]
[10, -5]
[-10, -5]
[-10, 0]
[0, -10]
[0, -15]
[10, 10]
[10, -15]
[0, 10]
[10, 15]
[10, 0]
[-10, 5]
[-10, 10]
[10, 5]
[0, 5]
[0, -5]
[-10, -15]
[0, 0]
[-10, -10]
[-10, 15]


In [132]:
current_files = glob.glob(os.path.join(IMAGE_PATHS[0], "00*_0P*"))
    
learning_indexes = np.random.choice(len(current_files), int(85/100*len(current_files)), replace=False)
testing_indexes = list(set(range(len(current_files))).difference(learning_indexes))

print(learning_indexes*100)
print(testing_indexes)


x = [2,3,5]
y = range(10)

list(np.asarray(y)[x])

[   0  200 1900  800 1500 1200  100 2000 1400  400 1600  300 1700  700
 1300  500 1800]
[6, 9, 10, 11]


[2, 3, 5]